Take from raw files - retr231/training/


Has SCOPe ID and secondary structure

Convert SCOPeID to PDB and add sequence and secondary structure -- save in one file

In [1]:
import os
from os import path
import pandas as pd
import glob
import csv

In [2]:
# Function to save different versions of the same pdb's:

def check_file(filePath):
    if path.exists(filePath):
        numb = 1
        while True:
            newPath = "{0}_{2}{1}".format(*path.splitext(filePath) + (numb,))
            if path.exists(newPath):
                numb += 1
            else:
                return newPath
    return filePath

In [3]:
os.chdir('/Users/Sophia/Dundee/dundee/retr231/training')

# Sequence
x = glob.glob('*.fasta')

# Prediction
# y = glob.glob('*.jnet')

for file in x:
    scop = file[:-6]
    pdb = scop[1:5]
    x_1 = open(file, 'r')
    x_2 = open(scop + '.jnet', 'r')
    a = x_1.read()
    b = x_2.read()
    
    df = pd.DataFrame([x.split(';') for x in a.split('\n')])
    df = df.iloc[:2]
    df2 = pd.DataFrame([x.split(';') for x in b.split('\n')])
    df2 = df2.iloc[1]

    df_f = pd.concat([df, df2], axis = 1)

    df_f['SCOPeID'] = df_f.iloc[0][0][1:]

    df_f[0] = df_f[0][1]
    df_f[1] = df_f[1][0]
    df_f = df_f.drop_duplicates()
    df_f[1] = df_f[1][0][9:]
    df_f[1]=df_f[1].str.replace(',','')
    
    df_f.columns = ['J_AA Sequence', 'J_SS', 'SCOPeID']
    df_f['PDB_ID'] = pdb
    df_f.reindex = ['PDB_ID', 'SCOPeID', 'J_SS', 'J_AA Sequence']

    filename = check_file('/Users/Sophia/Dundee/dundee/data/Jpred/jpred/%s.csv' % pdb)
    df_f.to_csv(filename)

In [4]:
# Merge
os.chdir('/Users/Sophia/Dundee/dundee/data/Jpred/jpred')
x = glob.glob('*.csv')

df = pd.concat(
    map(pd.read_csv, [file for file in x]), ignore_index=True)

df = df.iloc[:, 1:]
df = pd.DataFrame(df)
df = df[['PDB_ID', 'SCOPeID', 'J_SS', 'J_AA Sequence']]
print(df)
df.to_csv('/Users/Sophia/Dundee/dundee/data/Jpred/jpred_final.csv')



     PDB_ID  SCOPeID                                               J_SS  \
0      3sxu  d3sxua_  --EEEEE---------HHHHHHHHHHHHHHHHHH---EEEEE--HH...   
1      3f42  d3f42a_  ---HHHHHHHHHHHHHHHHHHHHHHH-EEEEEEE---EEEEEEE--...   
2      2lig  d2liga_  ----EEEEH---HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH...   
3      2pu9  d2pu9a_  --HHHHHHHHHHHHHHH---EE---HHHHHHHHHHHHHHHHHH---...   
4      1wny  d1wnya_  ---EEEEEEE----------EEEEEEE------H--HEEE------...   
...     ...      ...                                                ...   
1343   3mqd  d3mqda1  ---EEEEEEEEEE-----HHHHHHHHHH------------------...   
1344   4ead  d4eada3  ---EEEEEEE----EEEEE-HHHHHHHHHHH---------------...   
1345   1kpt  d1kpta_  -----------------HHHHHHHHH-------------EEEEE--...   
1346   1kqp  d1kqpa_  ---HHHHH---------HHHHHHHHHHHHHHHHHH-----EEEE--...   
1347   2ebs  d2ebsa1  --EEEE------EEEEEEEE-----EEEEEE---EEEEEE----EE...   

                                          J_AA Sequence  
0     KNATFYLLDNDTTVDGLSAVEQLVCEIAAERWRSG